# To see wether Overnorm changes hallucination 

1. Setup & Imports
2. Base Model Loading
3. Dataset Preparation
4. Over-Normalization Probe Set (Set A)
5. Hallucination Evaluation Set (Set B)
6. Model Variants
7. Evaluation Pipeline
8. Results & Analysis
9. Conclusions

In [1]:
%pip install transformers datasets accelerate torch sentence-transformers faiss-cpu evaluate

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.4.2-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached filelock-3.20.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.4.0-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached pyyaml-6.0.3-cp313-cp313-win_amd64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp313-cp313-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install unsloth

  Using cached protobuf-6.33.2-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached bitsandbytes-0.49.0-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached pillow-12.0.0-cp313-cp313-win_amd64.whl.metadata (9.0 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached bitsandbytes-0.49.0-py3-none-win_amd64.whl (54.7 MB)
Using cached peft-0.18.0-py3-none-any.whl (556 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 856.6 kB/s eta 0:0


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

c:\Code\06_llms\HorusLLM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"  # or mistral, qwen, etc.

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)
model_base.eval()
model_base.config.use_cache = True